In [53]:
import ee
import geemap

In [54]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [55]:
point = ee.Geometry.Point([-122.4439, 37.7538])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2') \
    .filterBounds(point) \
    .filterDate('2018-07-11', '2019-06-30') \
    .sort('ALGORITHM_VERSION') \
    .first() \
    .select('NO2_column_number_density')

vis_params = {
    'min': 0,
    'max': 0.0002,
    'bands': ['NO2_column_number_density'],
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Sentinel-5P TROPIMI NO2")

In [56]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2018-07-11'

In [57]:
image.get('CLOUD_COVER').getInfo()

In [58]:
nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image.geometry())
Map.addLayer(nlcd, {}, 'NLCD')
Map

Map(center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['position'], widget=HBox…

In [59]:
# Make the training dataset.
points = nlcd.sample(**{
    'region': image.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)

In [60]:
print(points.size().getInfo())

2411


In [61]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-111.8366243565994, 34.87228244934362]}, 'id': '0', 'properties': {'landcover': 52}}


In [62]:
# Use these bands for prediction.
bands = ['NO2_column_number_density']


# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [63]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'NO2_column_number_density': 7.956137415021658e-05, 'landcover': 52}}


In [64]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

Map(center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['position'], widget=HBox…

In [65]:
class_values = nlcd.get('landcover_class_values').getInfo()
class_values

[11,
 12,
 21,
 22,
 23,
 24,
 31,
 41,
 42,
 43,
 51,
 52,
 71,
 72,
 73,
 74,
 81,
 82,
 90,
 95]

In [66]:
class_palette = nlcd.get('landcover_class_palette').getInfo()
class_palette

['476ba1',
 'd1defa',
 'decaca',
 'd99482',
 'ee0000',
 'ab0000',
 'b3aea3',
 '68ab63',
 '1c6330',
 'b5ca8f',
 'a68c30',
 'ccba7d',
 'e3e3c2',
 'caca78',
 '99c247',
 '78ae94',
 'dcd93d',
 'ab7028',
 'bad9eb',
 '70a3ba']

In [67]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [68]:
Map.addLayer(landcover, {}, 'Land cover')
Map

Map(center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['position'], widget=HBox…

In [69]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [70]:
#Map.add_legend(builtin_legend='NLCD')
#Map

In [71]:
import os
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'landcover.tif')

In [72]:
geemap.ee_export_image(landcover, filename=out_file, scale=900)

Generating URL ...
An error occurred while downloading.
Total request size (52676208 bytes) must be less than or equal to 33554432 bytes.


Export the result to Google Drive:

In [73]:
geemap.ee_export_image_to_drive(landcover, description='landcover', folder='export', scale=900)

Exporting landcover ...


In [74]:
print(type(nlcd))

<class 'ee.image.Image'>


In [75]:
from IPython.display import Image, display

In [76]:
image